In [1]:
import camelot
import pandas as pd

In [2]:
pdf_path = "file.pdf"
tables = camelot.read_pdf(pdf_path, pages="23-49")

In [3]:
res = pd.concat(list(map(lambda x: x.df, tables)))
res.columns = ('Descripción', 'Unidad', 'Precio', "$")
res = res.reset_index(drop=True)
res["Tipo"] = pd.NA

In [4]:
regex = r'(?P<Unidad>\w+)\s(?P<Costo>[\d\.,]+)\s?(?P<az>\$)?\s(?P<Descripción>.+)'
view = res.loc[(res[["Unidad", "Precio"]] == ["", ""]).all(axis=1)].copy(deep=False)
mask = view["Descripción"].str.match(regex)
buff = view.loc[mask, "Descripción"].str.extract(regex).drop(columns="az")
view.loc[mask, ["Descripción", "Unidad", "Precio"]] = buff
res

,Descripción,Unidad,Precio,$,Tipo
0,EXTINTOR CONTRA INCENDIO,,,,NaN
1,EXTINTOR C.I.POLVO QUIMICO SECO ABC 1 KG,PZA,"20,00",$,NaN
2,EXTINTOR C.I.POLVO QUIMICO SECO ABC 2 KG,PZA,"30,00",$,NaN
3,EXTINTOR C.I.POLVO QUIMICO SECO ABC 4 KG,PZA,"35,00",$,NaN
4,EXTINTOR C.I.POLVO QUIMICO SECO ABC 6 KG,PZA,"50,00",$,NaN
...,...,...,...,...,...
3719,MEZCLADORA DE LAVADERO BAR MONOCOMANDO AL MUEB...,PZA,381.27,,NaN
3720,MEZCLADORA DE LAVADERO AL MUEBLE CRISTAL PICO ...,PZA,272.8,,NaN
3721,MEZCLADORA DE LAVADERO A LA PARED CRISTAL PICO...,PZA,304.15,,NaN
3722,MEZCLADORA DE LAVADERO AL MUEBLE MARES EGEO PI...,PZA,283.81,,NaN


In [5]:
mask = (res[['Unidad', 'Precio']] == ['', '']).all(axis=1)
dummy = res.copy(deep=False)
dummy.loc[mask, 'Tipo'] = dummy.loc[mask, 'Descripción']
res['Tipo'] = res['Tipo'].ffill()
res = res.reset_index(drop=True)
res

,Descripción,Unidad,Precio,$,Tipo
0,EXTINTOR CONTRA INCENDIO,,,,EXTINTOR CONTRA INCENDIO
1,EXTINTOR C.I.POLVO QUIMICO SECO ABC 1 KG,PZA,"20,00",$,EXTINTOR CONTRA INCENDIO
2,EXTINTOR C.I.POLVO QUIMICO SECO ABC 2 KG,PZA,"30,00",$,EXTINTOR CONTRA INCENDIO
3,EXTINTOR C.I.POLVO QUIMICO SECO ABC 4 KG,PZA,"35,00",$,EXTINTOR CONTRA INCENDIO
4,EXTINTOR C.I.POLVO QUIMICO SECO ABC 6 KG,PZA,"50,00",$,EXTINTOR CONTRA INCENDIO
...,...,...,...,...,...
3719,MEZCLADORA DE LAVADERO BAR MONOCOMANDO AL MUEB...,PZA,381.27,,MEZCLADORA DE LAVADERO
3720,MEZCLADORA DE LAVADERO AL MUEBLE CRISTAL PICO ...,PZA,272.8,,MEZCLADORA DE LAVADERO
3721,MEZCLADORA DE LAVADERO A LA PARED CRISTAL PICO...,PZA,304.15,,MEZCLADORA DE LAVADERO
3722,MEZCLADORA DE LAVADERO AL MUEBLE MARES EGEO PI...,PZA,283.81,,MEZCLADORA DE LAVADERO


In [6]:
res["$"] = "$"
res = res.dropna(axis=0)
res = res.loc[(res[["Unidad", "Precio"]] != ["", ""]).all(axis=1)]
res["Precio"] = res["Precio"].str.replace(",", ".").astype(float)

In [7]:
res.to_csv("materiales.csv")